# 1. Introducción

Este cuaderno tiene como objetivo demostrar el proceso que se necesita hacer si se requiere importar un pk12 a un JKS. Se utiliza el módulo de python pyopenSSL y JAVA que ya viene instalado en los linux de colab.

El estandar PKCS #12 define el intercambio de Sintexis de Información Personal. Se define un formato de archivo que se suele utilizar para almacenar llaves privadas junto con su respectivo certificado de llave publica protegidos bajo una llave simétrica basada en contraseña. Este archivo puede ser utilizado para firmar transacciones electrónicamente.

Un Java KeyStore (JKS) es un repositorio de certificados de seguridad. Puede contener certificados autorizadores o de llave pública.

Primero debemos crear un JKS utilizando el comando keytool que trae JAVA instalado en linux del entorno. Luego generamos un certificado con su respectiva key y formamos el pk12 con ellos. Lo ultimo es importar el pk12 al JKS y verifiar su correcta importación.


# 2. Armado del ambiente




##2.1 Instalación del paquete OpenSSl 

In [1]:
!pip install pyOpenSSl

     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 2.6MB 10.9MB/s 


##2.2 Creación de Keystore JKS

In [6]:
# --------------------------------------------
#@title 2.2.1 Parámetros del JKS { vertical-output: true }
Nombre_JKS =   "keystoreEA3"#@param {type:"string"}
Autor_JKS =   "FranciscoPretto"#@param {type:"string"}
Alias_JKS =   "ea3"#@param {type:"string"}
Password_JKS= "123qweqwe" #@param{type:"string"}

if(len(Nombre_JKS)<3 or len(Autor_JKS)<3 or len(Autor_JKS)<3):
    raise Exception("Los parametros deben tener al menos 3 caracteres")
if(len(Password_JKS)<6):
  raise Exception("La contraseña debe tener al menos 6 caracteres")
# --------------------------------------------

#Se genera el dname completo
cn=Autor_JKS+'.alumnos.unlam.edu.ar'
dname = 'CN='+cn+',OU=alumnos,O=unlam.edu.ar,L=LaMatanza,S=BuenosAires,C=AR'

#Se crea el JKS con una clave resguardada temporal
!keytool -genkey -noprompt -dname {dname} -alias {Alias_JKS} -keypass {Password_JKS} \
-storepass {Password_JKS} -keystore {Nombre_JKS}.jks

#Se elimina la clave temporal
!keytool -delete -alias {Alias_JKS} -storepass {Password_JKS} -keystore {Nombre_JKS}.jks

#Se muestra que esta vacio
!keytool -list -storepass {Password_JKS} -keystore {Nombre_JKS}.jks
!ls -l | grep *.jks

#Se utilizara para asignar numeros de serie
num_serie=0


Keystore type: PKCS12
Keystore provider: SUN

Your keystore contains 0 entries

-rw-r--r-- 1 root root   88 Dec  2 21:42 keystoreEA3.jks


# 3. Desarrollo




##3.1 Se genera un certificado para importar al JKS.

In [5]:
# --------------------------------------------
#@title 3.1.1 Parámetros del Certificado { vertical-output: true }

Filename =   "prettoEA"#@param {type:"string"}

if(len(Filename)<3):
    raise Exception("El filename debe contener al menos 3 caracteres")
# --------------------------------------------

import OpenSSL

#Asignación de numeros de Serie
num_serie += 1

#Se genera la key

key = OpenSSL.crypto.PKey()
key.generate_key(OpenSSL.crypto.TYPE_RSA, 2048)

#Se genera el certificado con sus atributos
cert = OpenSSL.crypto.X509()
cert.set_version(3)
cert.set_serial_number(num_serie)
cert.get_subject().CN = cn
cert.gmtime_adj_notBefore(0)
cert.gmtime_adj_notAfter(365)
cert.set_issuer(cert.get_subject())
cert.set_pubkey(key)

#Se firma el certificado
cert.sign(key, "sha256")

#Se guardan los archivos del certificado (.cer) y la llave privada (.key)
open(Filename+".cer", "wb").write(
               OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, cert))

open(Filename+".key", "wb").write(
               OpenSSL.crypto.dump_privatekey(OpenSSL.crypto.FILETYPE_PEM, key))

#Se muestra el certificado y la key

!echo -e "Certificado:\n"
!cat "{Filename}".cer
!echo -e "\n"
!ls -l | grep *.cer
!echo -e "\nKey:\n"
!cat "{Filename}".key
!echo -e "\n"
!ls -l | grep *.key


Certificado:

-----BEGIN CERTIFICATE-----
MIIC1zCCAb+gAwIBAwIBATANBgkqhkiG9w0BAQsFADAvMS0wKwYDVQQDDCRGcmFu
Y2lzY29QcmV0dG8uYWx1bW5vcy51bmxhbS5lZHUuYXIwHhcNMjAxMjAyMjE0MjMx
WhcNMjAxMjAyMjE0ODM2WjAvMS0wKwYDVQQDDCRGcmFuY2lzY29QcmV0dG8uYWx1
bW5vcy51bmxhbS5lZHUuYXIwggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIB
AQDe9vPexHoaYj6E/fmVi8WW5AgzScaaIP41If8OvA3zD0Qk9Yw3G6YNr3TGVH1F
L/lBX/iyBiFFu+nN2BYs5vFu3XwGkKF99ZZ++QCbTytgi6EZ5+9Z7rIUUm5+cDds
sr34JT/mWeeOlhlyGv3dabjxaS6EB583E0yhoSiQP3ZMBD04/KG7PkB8zAOwYkXA
X3N3+LmxEk4x+u5H4Uy/+JHjuL0rMBlrk7lCwRQD9J7mLLkaSaCY6m4BelXn4lu8
WWH8uwcDYorpFSZ5YOGaClAfSoHhAoiXmo3Hc1MPmmEfaJaVnCiSRzHNfzQ27/2S
Gi5OOGA8k1zUniP4Xn8GykB/AgMBAAEwDQYJKoZIhvcNAQELBQADggEBAGrpgwpW
l4cmYdzkcJtNHnKyzwNqmXw0Dc1QP1DxXxiDC0ibedKj5lHuoKXYLayIUOMg0l+p
+dbN0aNjgLKazTRTP2MKBqfYJGXlh3GmRnPyRaIEE0FCO/eFoglV4j9sKiDf0jSg
wVXz3XLITfluxE4WqmsLHBDOyrfbXNVEWzmy8dfgx/KWqnBOdCQ144QnTeo4W5fx
bjkBZnXRuqtjv6+cNOvGTEWj64iL/7Qzfyo9m3dLz/nIRtQXVjAq6twPHlxeE1HA
fb8J/zUEb5rJYP//ugEsmO8aANiJulsjhYPPIdFOFXntTvWt

##3.2 Se genera el PKCS12

In [7]:
# --------------------------------------------
#@title 3.2.1 Parámetros del PK12 { vertical-output: true }
Alias_PK12 =   "prettoEA"#@param {type:"string"}
Password_PK12 =   "hola1234"#@param {type:"string"}

if(len(Alias_PK12)<3):
    raise Exception("El Alias debe tener al menos 3 caracteres")
if(len(Password_PK12)<6):
  raise Exception("La contraseña debe tener al menos 6 caracteres")
# --------------------------------------------

#Se genera el pk12
p12 = OpenSSL.crypto.PKCS12()

#Se setea la key y el certificado
p12.set_privatekey(key)
p12.set_certificate(cert)
p12.set_friendlyname(Alias_PK12.encode("utf-8"))

filename_pfx=Filename+'.pfx'
#Se guarda el archivo colocandole la contraseña correspondiente
with open(filename_pfx,'wb') as file:
        file.write(p12.export(passphrase=Password_PK12.encode("utf-8")))

!ls -l | grep *.pfx

-rw-r--r-- 1 root root 2394 Dec  2 21:43 prettoEA.pfx


##3.3 Se importa el PK12 al JKS

In [9]:
#Comando para importarlo
!keytool -v -importkeystore -noprompt -srckeystore {Filename}.pfx -srcstoretype PKCS12 -srcstorepass {Password_PK12} -destkeystore {Nombre_JKS}.jks -storepass {Password_JKS} -deststoretype JKS

#Se muestra el pk12 importado
!echo -e "\nSe listan los certificados en el JKS:\n"
!keytool -list -storepass {Password_JKS} -keystore keystoreEA3.jks

Importing keystore prettoEA.pfx to keystoreEA3.jks...
Entry for alias prettoea successfully imported.
Import command completed:  1 entries successfully imported, 0 entries failed or cancelled
[Storing keystoreEA3.jks]

Se listan los certificados en el JKS:

Keystore type: PKCS12
Keystore provider: SUN

Your keystore contains 1 entry

prettoea, Dec 2, 2020, PrivateKeyEntry, 
Certificate fingerprint (SHA-256): DE:32:4B:FA:63:06:C8:96:DD:E9:16:1F:69:17:0B:4F:0F:E1:FE:D5:C0:E9:83:D4:35:A1:52:85:D7:1C:40:CD


# 4. Tabla de pasos

> La ejecución de este cuaderno se realiza plenamente en el CPU


 Entorno| Función | Detalle
---------|----------|--------
Linux | pip install pyOpenSSl| Instala openSSL para python
Python|@param| Lee parametros desde el cuaderno
Linux| keytool -genkey| Genera el JKS Keystore
Linux| keytool -delete| Elimina una clave resguardada
Linux| keytool -list | Lista claves
Linux | ls -l | Lista en detalle elementos en un directorio
Linux | grep | Filtra
Python|import OpenSSL| Importa el modulo previamente instalado
Python|OpenSSL.crypto.PKey()| Crea el objeto key
Python | generate_key(OpenSSL.crypto.TYPE_RSA, 2048) | Genera la key RSA 2048
Python|OpenSSL.crypto.X509() | Crea el objeto certificado
Python|set_version(3) | Setea la version *
Python|set_serial_number(num_serie) | Asigna número de serie
Python|get_subject().CN = cn | Setea common name
Python|gmtime_adj_notBefore(0) | Ajusta desde cuando el certificado empieza a ser valido
Python|gmtime_adj_notAfter(365) |  Ajusta hasta cuando el certificado es valido
Python|set_issuer(cert.get_subject()) | Setea el issuer. Entidad que verifica el contenido del cert
Python|set_pubkey(key) | Setea la key publica utilizando la previamente creada
Python|cert.sign(key, "sha256")|Se firma al certificado con la key privada en SHA256
Python|open(Filename+".cer", "wb").write(OpenSSL.crypto.dump_certificate|Crea el archivo .cer
Python|open(Filename+".key", "wb").write(OpenSSL.crypto.dump_privatekey|Crea el archivo .key
Linux| echo -e |Imprime texto en la pantalla
Linux|cat| imprime contenido de un archivo en pantalla
Python|OpenSSL.crypto.PKCS12()| Genera el pk12
Python|set_privatekey(key)| Setea la privada
Python|set_certificate(cert)| Setea el certificado
Python|set_friendlyname(Alias_PK12.encode("utf-8"))| Setea el Alias del pk12
Python|open(filename_pfx,'wb') as file:file.write(p12.export(passphrase=Password_PK12.encode("utf-8")))| Se exporta el pfx con la password inidcada
Linux|keytool -v -importkeystore| Importa el pfx al JKS

\* Las versiones son 1, 2 y 3. En cada una se fueron introduciendo datos. En la versión 2 se introducieron el issuer y el subjet. En la versión 3 se introducieron las extensiones.


# 5. Conclusiones

El módulo <strong>pyOpenSSL</strong> es muy útil cuando se necesita automatizar el proceso de la generación de pk12 pero no se encontro una forma práctica para importarlos desde Python. Por otro lado, hay que tener en cuenta que este caso es simplificado debido a que el script esta corriendo sobre el equipo que tiene el JKS y no debemos conectarnos por SSH a otro equipo para poder importar el pfx. Se debe agregar la complejidad de verificar con que usuario se debe realizar la creación / importacion en el JKS y en que path estos se encuentran.

De todas maneras, un script con este módulo puede ahorrar muchas horas hombre y evitar errores si se desarrolla con las precauciones antes explicadas.

Este cuaderno se puede separar en 4 pasos:

1.   Generar el JKS
2.   Generar un Certificado con su Key privada
3.   Generar el pk12 en formato pfx
4.   Importar el pk12 al keystore.

Los puntos 2 y 3 son los que se pudieron desarrollar con pyOpenSSl, pero la interacción con el keystore se tuvo que realizar por comandos de linux debido a que era la forma más práctica de realizar las tareas.

Aprender a utilizar el modulo de OpenSSL tomo tiempo pero luego uno puede ver que es similar en como se generan y rellenan los distintos objetos. Hay varios detalles que no tenia en cuenta cuando empece, algunos datos, como la contraseña del pfx debe estar encodeada en utf-8, se realizo ejecutando <strong>"encode("utf-8")"</strong>.

Por otro lado, se debe crear el archivo abriendolo como "wb" ( en modo binario), sino da un error que parece enviarte a la dirección contraria... "TypeError: write() argument must be str, not bytes".

Mas allá de estos detalles, el módulo es facil si tenemos conocimientos en certificados.

Una mejora evidente seria poder crear e importar en el JKS directamente desde python y ver una forma de importar el pk12 creado en varios equipos. Asi mismo, tambien se podria encontrar una forma de importar las contraseñas de los JKS desde una bóveda de contraseñas.





#6. Bibliografía

Se utilizaron los apuntes de la catedra y las siguientes páginas web:

*   https://www.ssl.com/es/c%C3%B3mo/crear-un-archivo-de-certificado-pfx-p12-usando-openssl/
*   http://www.firmaelectronica.chiapas.gob.mx/sitio/glosario#:~:text=El%20est%C3%A1ndar%20PKCS%20%2312%20define,llave%20sim%C3%A9trica%20basada%20en%20contrase%C3%B1a.
* https://kb.stl4me.com/herramientas/importar-p12-a-jks-con-keytool/
* https://stackoverflow.com/questions/60837051/how-to-load-a-pkcs12-keystore-using-python
* https://es.stackoverflow.com/questions/211158/generar-un-certificado-cert-y-key
*https://nodored.com/clientes/knowledgebase/330/Crear-un-archivo-de-certificado-.pfxor.p12-utilizando-OpenSSL.html
*https://stackoverflow.com/questions/17935619/what-is-difference-between-cacerts-and-keystore
*https://stackoverflow.com/questions/15964797/unable-to-import-p12-certificate-to-cacerts
*https://www.javaer101.com/es/article/173348.html
*https://stackoverflow.com/questions/62901667/how-to-create-a-jks-or-p12-keystore-with-python
*https://stackoverrun.com/es/q/4057596
*https://security.stackexchange.com/questions/45996/sslv3-or-ssl-certificate-version-2
*https://stackoverflow.com/questions/33679936/using-pyopenssl-to-generate-p12-pfx-containers
*https://github.com/pyca/pyopenssl/issues/681


